In [2]:
import lightgbm as lgb
from sklearn import metrics
import category_encoders as ce

from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_csv('../data/train.csv',dtype={'tipodepropiedad':'category','ciudad':'category','provincia':'category'})
df['fecha'] = pd.to_datetime(df['fecha'])
df['anio'] = df['fecha'].dt.year
df["mes"] = df['fecha'].dt.month
df["dia"] = df['fecha'].dt.day

df = df.drop(columns=["lat","lng"])

# Funciones a utilizar

In [4]:
def imputar(df,cols):
    imp = SimpleImputer()
    df_result = df.copy()
    for x in cols:
        df_result[x]=imp.fit_transform(df[[x]])
    return df_result

In [5]:
def normalizar(df):
    min_max_scaler = MinMaxScaler()
    cols = df._get_numeric_data().columns
    sub_df = df[cols]
    res = df.drop(columns=cols)
    x_scaled = min_max_scaler.fit_transform(sub_df)
    x_scaled = pd.DataFrame(x_scaled,columns=cols)
    return res.join(x_scaled)

In [6]:
import re
def contiene_frase(texto, frase):
    if re.search(frase, texto,re.IGNORECASE):
        return True
    return False

contiene_frase=np.vectorize(contiene_frase)

In [7]:
def cantidad_de_public(data,colum,palabra):
    return data[contiene_frase(data[colum],palabra)].dia.count()

def serie_que_contiene_palabra(data,colum,palabra):
        return contiene_frase(data[colum],palabra)

## Completo los datos faltantes

In [8]:
imp = SimpleImputer(strategy="constant")

df["provincia"] = imp.fit_transform(df[["provincia"]])
df["ciudad"] = imp.fit_transform(df[["ciudad"]])

In [9]:
df['ciudad-provincia'] = df[['ciudad', 'provincia']].apply(lambda x: ', '.join(x), axis=1)

In [10]:
#Columnas numericas
data_cols = df._get_numeric_data().columns

#Completo los nulos
df = imputar(df,data_cols)


#Normalizo pero --> Hay veces que no me conviene

#df = normalizar(df)

 ## Genero los encodes


In [11]:
oneHot = ['tipodepropiedad','provincia']
label = ['ciudad','ciudad-provincia']

encoder = ce.OneHotEncoder()

encoded = encoder.fit_transform(df[oneHot])
data = df.join(encoded)

encoder = ce.BinaryEncoder()

encoded = encoder.fit_transform(df[label])
data = data.join(encoded)

In [12]:
df = data

In [30]:
df.to_csv("../data/dataset_imputado.csv", index=False)